#
### Python Preamble

In [1]:
# Analysis of Disulfide Bonds in Proteins of Known Structure 
# Author: Eric G. Suchanek, PhD.
# Last revision: 1/19/23 -egs-
# Cα Cβ Sγ

import math
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import plotly_express as px
#import seaborn as sns

import proteusPy
from proteusPy import *
from proteusPy.Disulfide import Disulfide
from proteusPy.DisulfideLoader import Load_PDB_SS
from proteusPy.DisulfideList import DisulfideList

from proteusPy.ProteusGlobals import *

import pandas as pd
import pyvista as pv
from pyvista import set_plot_theme

#print(pv.Report())

plt.style.use('dark_background')

# ipyvtklink
# pv.set_jupyter_backend('ipyvtklink')

set_plot_theme('dark')

# the locations below represent the actual location on the dev drive.
# location for PDB repository
PDB_BASE = '/Users/egs/PDB/'

# location of cleaned PDB files
PDB = '/Users/egs/PDB/good/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_BASE}data/'


# Analysis of Disulfide Bonds in Proteins Within the RCSB Protein Data Bank
*Eric G. Suchanek, PhD. (suchanek@mac.com)* <br> <br>

## Summary
I describe the results of a structural analysis of Disulfide bonds contained in 35,819 proteins within the RCSB Protein databank, https://www.rcsb.org. These protein structures contained 120,697 Disulfide Bonds.  The analysis utilizes Python functions from my ``ProteusPy`` package https://github.com/suchanek/proteusPy/, which is built upon the excellent ``BioPython`` library (https://www.biopython.org). 

## Background
This work represents a reprise of my original Disulfide modeling analysis conducted in 1986 ([publications](#publications) item 1) as part of my doctoral dissertation. Given the original Disulfide database contained only 2xx Disulfide Bonds I felt it would be interesting to revisit the RCSB and mine the thousands of new structures. The analysis would not have been possible withoug the creation and now ongoing development of ```proteusPy```, which represents a true, modern object-oriented rewrite of my original ```proteus``` code base. The ```C``` implemntation still exists and is available at https://github.com/suchanek/proteus/.

The initial results are shown and described in the cells below.

## Intended Audience
This notebook is intended primarily for people interested in structural biophysics with an emphasis on the analysis of protein structures. As a result, it is fairly advanced. I have made an effort to describe the analysis plainly, but I assume the reader has a basic understanding of the elements of protein structure.

## Requirements
 - My Biopython fork or my delta applied, available at: https://github.com/suchanek/biopython/
 - proteusPy: https://github.com/suchanek/proteusPy/

## Introduction
Disulfide bonds are important covalent stabilizing elements in proteins, and function as intra-molecular cross-bridges. They are formed when two Sulphur-containing Cysteine (Cys) amino acid residues are close enough and in the correct geometry to form a S-S covalent bond with their terminal sidechain $S_\gamma$ atoms. The resulting residue is known as *Cystine*, or more commonly a *Disulfide* bond. This cross-bridge stabilizes the connected protein backbone via the alpha carbon, ($C_\alpha$) backbone atoms with this strong S-S covalent bond. This bond has high energy barriers to rotation, and as such has manifests partial double-bond character. Disulfide bonds most commonly occur between alpha helices and greatly enhance a protein's stability to denaturation. 


In [2]:
# default parameters will read from the package itself.
PDB_SS = Load_PDB_SS(verbose=True, subset=False)
PDB_SS.describe()



-> load_PDB_SS(): Reading /Users/egs/repos/proteusPy/proteusPy/data/PDB_SS_ALL_LOADER.pkl... done.
========= RCSB Disulfide Database Summary 0.51dev =========
PDB IDs present: 35818
Disulfides loaded: 120697
Total RAM Used by dataset: 29.26 GB.
Best Disulfide: <Disulfide 2q7q_75D_140D, Source: 2q7q, Resolution: 1.6 Å Χ1-Χ5: -59.36°, -59.28°, -83.66°, -59.82° -59.91°, 0.49 kcal/mol
Worst Disulfide: <Disulfide 1toz_456A_467A, Source: 1toz, Resolution: None Å Χ1-Χ5: 107.16°, 111.29°, -13.24°, 110.70° 100.94°, 18.12 kcal/mol


In [5]:
# display the best and worst SS
ssMin, ssMax = PDB_SS.SSList.minmax_energy()
minmaxlist = DisulfideList([ssMin, ssMax], 'mm')
minmaxlist.display(style='bs', light=True)


ViewInteractiveWidget(height=512, layout=Layout(height='auto', width='100%'), width=1024)

In [3]:
best = PDB_SS.get_by_name('2q7q_75D_140D')
worst = PDB_SS.get_by_name('1toz_456A_467A')
duo = DisulfideList([best, worst], 'bw')
duo.display(style='sb')

ViewInteractiveWidget(height=512, layout=Layout(height='auto', width='100%'), width=1024)

## Examine the Disulfide $C_\alpha-C_\alpha$ Distances

The disulfide bond maximum $C_\alpha-C_\alpha$ distance is constrained by the bond lengths and bond angles of the cystine (Cys-Cys) sidechain atoms. If the bond angles were linear the maximum possible $C_\alpha-C_\alpha$ distance would be 8.82 $\AA$. We can examine the database easily with Pandas to see this distance distribution.

In [6]:

# retrieve the torsions dataframe
from proteusPy.Disulfide import Torsion_DF_Cols

_SSdf = PDB_SS.getTorsions()

# there are a few structures with bad SSBonds. Their
# CA distances are > 7.0. We remove them from consideration
# below

_far = _SSdf['ca_distance'] >= 9.0
_near = _SSdf['ca_distance'] < 9.0

SS_df_Far = _SSdf[_far]

# entire database with near cutoff of 9.0
SS_df = _SSdf[_near]
SS_df = SS_df[Torsion_DF_Cols].copy()

SS_df_Far.describe()


,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist,torsion_length
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,157.551724,308.386207,-30.656662,36.914964,-4.246220,-22.020778,-28.570280,7.126051,36.713703,-96.615169,42.827219,-98.954139,39.183479,219.825736
std,250.000998,939.934212,86.894645,89.968912,93.231625,100.932830,111.258425,2.553747,21.468687,48.116280,119.643862,45.484317,122.094933,43.486354
min,1.000000,1.000000,-179.824424,-168.391245,-179.734548,-176.487239,-179.807853,2.190514,9.124401,-180.000000,-180.000000,-180.000000,-180.000000,77.516080
25%,14.000000,45.000000,-71.267337,-0.323513,-62.353988,-86.761608,-75.611442,5.004464,20.149808,-125.561892,-36.750426,-127.290605,-41.277690,191.084559
50%,50.000000,85.000000,-62.761294,21.892198,-6.647887,-65.127046,-62.924252,7.006149,29.165195,-73.574817,125.909647,-96.504329,122.883495,214.237201
75%,156.000000,333.000000,-46.810535,119.156344,48.308780,58.841591,44.429564,8.804118,51.029830,-64.481368,141.919824,-61.101131,131.723006,253.454460
max,1498.000000,8774.000000,179.681323,179.033585,177.542951,164.514396,179.742066,14.761692,101.701154,170.612433,171.910637,84.092757,179.681140,306.047948


Initial analysis shown above reveals 145 physically impossible disulfides. These will be removed from consideration by rejecting disulfides whose $C_\alpha-C_\alpha$ distances are $> 9 \AA$. This should improve our overall protein structure data quality.


In [7]:
SS_df.describe()

,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist,torsion_length
count,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000,120552.000000
mean,225.033421,273.124253,-46.595878,-15.334268,-2.470160,-27.627378,-38.697196,3.542794,5.521266,-97.819642,62.214192,-96.175434,63.949380,224.181947
std,272.602074,274.891270,97.029275,102.464276,93.912092,103.715589,94.140418,2.357670,0.760387,43.798455,99.000135,43.529300,96.404402,52.608647
min,1.000000,1.000000,-179.998217,-179.999554,-179.981786,-179.999510,-179.999305,0.491737,2.831762,-180.000000,-180.000000,-180.000000,-180.000000,91.894909
25%,47.000000,96.000000,-83.436638,-87.701731,-87.404378,-92.808351,-75.112726,1.909088,5.076505,-128.640539,-27.957661,-123.288007,-24.241389,181.135515
50%,136.000000,193.000000,-63.668075,-58.616963,-64.576892,-66.491796,-61.044216,3.020678,5.592829,-96.121921,112.917184,-97.430376,115.183472,225.218200
75%,305.000000,361.000000,-45.096830,80.988702,94.713461,73.642308,-14.233589,4.345451,6.072908,-68.943415,143.688837,-70.605424,143.217378,261.541528
max,4374.000000,4443.000000,179.999749,179.996783,179.995070,179.999222,179.999693,18.124530,8.498543,179.912372,179.999324,179.849134,179.993389,381.310941



When we filter by distance, we see the average $C_\alpha-C_\alpha$ distance for the entire dataset is 5.52 $\AA$ , with a minimum distance of 2.83 $\AA$ and a maximum of 8.50 $\AA$. The 145 disulfides whose had distances >= 9 $\AA$ will be removed from consideration since they are not physically possible.

In [14]:
sslist = PDB_SS.SSList
ss = sslist.get_by_name('1twa_110A_148A')
ss

<Disulfide 1twa_110A_148A, Source: 1twa, Resolution: 3.2 Å>

In [13]:
# The distances are in the SS_df dataframe and can be sorted easily
SS_df.sort_values(by=['ca_distance'], ascending=False, inplace=True)

SS_df.head(10)


,source,ss_id,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist,torsion_length
84010,1twa,1twa_110A_148A,110,148,-1.415198,151.707385,-145.031081,172.634306,176.666409,10.464060,8.498543,-66.889869,-36.974047,-84.607875,82.874040,324.136964
89613,6z9g,6z9g_78B_492B,78,492,69.197315,-135.951748,-25.166881,173.060676,-83.875722,9.817678,8.391020,-120.412496,43.920969,-61.727501,-41.688624,246.758288
53986,1twh,1twh_110A_148A,110,148,-4.511875,157.074175,-144.457197,169.607473,170.154474,10.393050,8.390433,-72.205998,-43.783894,-78.118091,82.744757,321.371709
30567,7krs,7krs_480A_488A,480,488,-65.962832,-169.706675,39.259684,166.678286,-68.561165,5.186595,8.381247,-140.682847,122.923232,-74.646552,151.325447,259.181515
46235,5mdl,5mdl_78L_600L,78,600,59.109100,-134.280439,17.495379,148.495333,-96.520361,12.816469,8.354424,-84.661521,53.565937,-56.004656,-46.874397,230.647467
85528,6gal,6gal_79L_579L,79,579,58.319891,-129.998360,3.017870,155.663808,-88.654415,12.613903,8.353788,-84.921179,46.702387,-57.120893,-46.082226,228.912030
18069,3dll,3dll_36Y_46Y,36,46,46.344014,174.566430,152.625186,-167.008792,151.699307,8.612707,8.338130,-37.404963,-12.778341,-64.000613,115.980697,326.833704
94666,6ehq,6ehq_64L_549L,64,549,61.755538,-137.054258,15.811365,151.013678,-88.841901,11.947480,8.335290,-95.835840,55.683782,-58.853753,-50.573314,231.399286
8449,2atc,2atc_109B_140B,109,140,-141.073344,-154.780034,-8.550015,-123.377952,69.931059,13.888352,8.334744,-37.900494,115.855016,-139.533911,50.541633,253.069402
59944,6z7r,6z7r_78B_492B,78,492,71.567018,-133.711069,-24.156972,167.502919,-80.157903,9.938207,8.332175,-116.950924,45.454375,-65.179589,-36.295915,240.970049


We can extract and visualize the four longest structures and display them, as shown below. The individual windows display the disulfide bond in 'split-bond' style, where half of the bond is colored by the respective atom color. The window title indicates the approximate torsional energy (E kcal/mol), $C_\alpha-C_\alpha$ distance ($\AA$), and the torsion length (degrees). As is apparent, the $C_\beta-S_\gamma$ bond angles are almost linear. This suggests that the disulfide might not actually exist covalently, and could reflect errors in the original model.

In [9]:
# extract and display the 4 longest SS bonds

i = 0
quattro = DisulfideList([], 'L4')

for i in range(4):
    ssid = SS_df.iloc[i]['ss_id']
    ss = PDB_SS[ssid]
    quattro.append(ss)

quattro.display(style='sb', light=True)

    

DisulfideException: DisulfideLoader(): Cannot find key 1twa_110A_148A in SSBond dict!

Plotting $C_\alpha-C_\alpha$ distances

In [ ]:
#
labels = {'value': 'Cα Distance', 'variable': 'Count'}
cols = ['ca_distance']
data = SS_df[cols].copy()
#px.histogram(SS_df['ca_distance'], labels=labels, template='plotly_dark')
px.histogram(SS_df['ca_distance'], labels=labels)


## Examining Disulfide Energies
The approximate bond torsional energy (kcal/mol) of a given disulfide bond 
is calculated using the following equation: \n

$ E_{kcal/mol} \approx 2.0 * cos(3.0 * \chi_{1}) + cos(3.0 * \chi_{5}) + cos(3.0 * \chi_{2}) + $
$ cos(3.0 * \chi_{4}) + 3.5 * cos(2.0 * \chi_{3}) + 0.6 * cos(3.0 * \chi_{3}) + 10.1 $

The equation embodies the typical 3-fold rotation barriers associated with single bonds,
($\chi_1, \chi_5, \chi_2, \chi_4$) and a high 2-fold barrier for $\chi_3$, resulting from the partial double bond
character of the S-S bond. This property leads to two major disulfide families, characterized
by the sign of $\chi_3$. *Left-handed* disulfides have $\chi_3$ < 0° and *right-handed* disulfides have
$\chi_3$ > 0°.


In [ ]:
labels = {'value': 'Energy', 'variable': 'Count'}
cols = ['energy']
#px.histogram(SS_df['energy'], labels=labels, histnorm='probability', template='plotly_dark')
px.histogram(SS_df['energy'], labels=labels, histnorm='probability')


### Find the lowest and highest energy disulfides and display them

In [ ]:

All_SS_list = PDB_SS.SSList
ssMin, ssMax = PDB_SS.SSList.minmax_energy()
minmaxlist = DisulfideList([ssMin, ssMax], 'mm')
minmaxlist.display(style='bs', light=True)

In [ ]:
minmaxlist.display_overlay(light=True)
ssMin.pprint()

Very interesting result! Even though both disulfides span almost the exact same distance 5.5 $\AA$ vs 5.2 $\AA$ they do so with entirely different geometries. The minimum energy structure's torsion length of $145.62 \AA$ is significantly shorter than the maximum's $215.61 \AA$, despite the latter's *cis* $S_\gamma - S_\gamma$ bond. This *cis* conformation sits squarely on the disulfide's energy maximum, a fact reflected in the high 18.12 kcal/mol computed torsional energy.

On the other hand, *2q7q 75D-140D*, the lowest-energy disulfide is beautiful. Its conformation (-59.358°, -59.278°, -83.663°, -59.819° -59.914°) is only 7° from an overall global minimum ($\chi_3 = 90°$)

In [ ]:
ssMin.display(style='bs')

The structure above shows that the previous and next residues have missing atoms (the long vectors drawn).

In [ ]:
SS_df = _SSdf[_near]
SS_df = SS_df[Torsion_DF_Cols]

SS_df = SS_df.sort_values(by=['energy'])
ssid_list = SS_df['ss_id'].values

good_SS_list = DisulfideList([],'l')
bad_SS_list = DisulfideList([],'h')

ss = Disulfide('tmp')

# first 12 are lowest energy
for i in range(12):
    ssid = ssid_list[i]
    ss = PDB_SS.get_by_name(ssid)
    good_SS_list.append(ss)
    
for i in range(13):
    if i == 0:
        continue
    ssid = ssid_list[-i]
    ss = PDB_SS.get_by_name(ssid)
    bad_SS_list.append(ss)
    


In [ ]:
bad_SS_list.display_overlay(light=False)


#bad_SS_list.display(style='sb')


In [ ]:
labels = {'value': 'Distance', 'variable': 'Distance'}
cols = ['ca_distance', 'energy']
data = SS_df[cols].copy()
data.set_index('ca_distance')

px.histogram(SS_df, x='ca_distance', y='torsion_length', labels=labels, histfunc='avg', nbins=90, template='plotly_dark')


## Examine the Disulfide Torsions
The disulfide bond's overall conformation is defined by the sidechain dihedral angles $\chi_{1}$-$\chi_{5}$. Since the S-S bond has electron delocalization, it exhibits some double-bond character with strong minima at $+90°$ and $-90°$. The *Left-handed* Disulfides have $\chi_{3}$ < 0.0° and the *Right-handed* have a $\chi_{3}$ > 0.0°.

These torsion values along with the approximate torsional energy are stored in the DisulfideLoader() class and individually within each Disulfide object. We access them via the ``DisulfideList.getTorsions()`` function.

We can get a quick look at their overall statistics using the ``Pandas.describe()`` function.



In [ ]:
labels = {'value': 'Angle', 'variable': 'Torsion'}

px.histogram(SS_df, x='chi1', y='chi5', labels=labels, nbins=90, histfunc='avg',
            template='plotly_dark')


In [ ]:
labels = {'value': 'Angle', 'variable': 'Torsion'}

px.histogram(SS_df, x='chi2', y='chi4', labels=labels, nbins=90, histfunc='avg',
            template='plotly_dark')


### Examining torsions by Disulfide Handedness
We split the dataset into these two families easily with Pandas.

In [ ]:
# make two dataframes containing left handed and right handed disulfides

_left = SS_df['chi3'] <= 0.0
_right = SS_df['chi3'] > 0.0

# left handed and right handed torsion dataframes
SS_df_Left = SS_df[_left]
SS_df_Right = SS_df[_right]

# routine creates 2 lists  for left-handed and right-handed disulfides 

ss_list = PDB_SS.SSList
left_handed_SS = DisulfideList([], 'left_handed')
right_handed_SS = DisulfideList([], 'right_handed')

for ss in ss_list:
    if ss.chi3 < 0:
        left_handed_SS.append(ss)
    else:
        right_handed_SS.append(ss)

print(f'Left Handed: {len(left_handed_SS)}, Right Handed: {len(right_handed_SS)}')


In [ ]:
SS_df_Left.describe()

In [ ]:
SS_df_Right.describe()

We can use a hexbin plot to generate the equivalent of a heatmap to explore the Correlation for the Chi1-Chi5 and Chi2-Chi4 dihedral angles. Since the disulfide bond is symmetric about Chi3 (the S-S bond), one would expect the distribution of Chi1 to be similar to Chi5, and the same with Chi2 and Chi4.

In [ ]:

# Set the figure sizes and axis limits.
DPI = 220
WIDTH = 6.0
HEIGHT = 3.0
TORMIN = -179.0
TORMAX = 180.0
GRIDSIZE = 20


In [ ]:
# !!!

x = SS_df_Left['chi1']
y = SS_df_Left['chi5']

x2 = SS_df_Right['chi1']
y2 = SS_df_Right['chi5']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

ax1, ax2 = fig.subplots(1, 2, sharey=True)

fig.suptitle(r'$\chi_{1} - \chi_{5}$ Correlation', fontsize=8)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)

ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\chi_{1 (left-handed)}$')
ax1.set_ylabel(r'$\chi_{5}$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(TORMIN-1, TORMAX+1)

ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\chi_{1 (right-handed)}$')

ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()



The distributions look extremely similar for both the left-handed and right-handed populations. Let's look at Chi2 and Chi4 to see if this holds true.

In [ ]:
x = SS_df_Left['chi2']
y = SS_df_Left['chi4']

x2 = SS_df_Right['chi2']
y2 = SS_df_Right['chi4']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

fig.suptitle(r'$\chi_{2} - \chi_{4}$ Correlation', fontsize=8)

ax1, ax2 = fig.subplots(1, 2, sharey=False)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\chi_{2 (left-handed)}$')
ax1.set_ylabel(r'$\chi_{4}$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(-180, 180)
ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\chi_{2 (right-handed)}$')
ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()



The distributions for left/right show distinct differences. Both show the predicted minimum around -75 degrees, but the right-handed group shows a population of disfuldies in the +90 degree range.

In [ ]:
x = SS_df_Left['chi1']
y = SS_df_Left['chi3']

x2 = SS_df_Right['chi1']
y2 = SS_df_Right['chi3']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

fig.suptitle(r'$\chi_{1} - \chi_{3}$ Correlation', fontsize=8)

ax1, ax2 = fig.subplots(1, 2, sharey=False)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\chi_{1 (left-handed)}$')
ax1.set_ylabel(r'$\chi_{3}$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(-180, 180)
ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\chi_{1 (right-handed)}$')
ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()



In [ ]:
x = SS_df_Left['chi2']
y = SS_df_Left['chi3']

x2 = SS_df_Right['chi2']
y2 = SS_df_Right['chi3']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

fig.suptitle(r'$\chi_{2} - \chi_{3}$ Correlation', fontsize=8)

ax1, ax2 = fig.subplots(1, 2, sharey=False)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\chi_{2 (left-handed)}$')
ax1.set_ylabel(r'$\chi_{3}$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(-180, 180)
ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\chi_{2 (right-handed)}$')
ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()



In [ ]:
x = SS_df_Left['chi2']
y = SS_df_Left['chi3']

x2 = SS_df_Right['chi2']
y2 = SS_df_Right['chi3']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

fig.suptitle(r'$\chi_{4} - \chi_{3}$ Correlation', fontsize=8)

ax1, ax2 = fig.subplots(1, 2, sharey=False)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\chi_{4 (left-handed)}$')
ax1.set_ylabel(r'$\chi_{3}$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(-180, 180)
ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\chi_{4 (right-handed)}$')
ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()



In [ ]:
#

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)
ax1 = plt.subplot(111)

x = SS_df['chi3']
y = SS_df['energy']

ymax = 10.0
plt.hexbin(x, y, gridsize=40, cmap='nipy_spectral')
ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(y.min(), ymax)
ax1.set_xlabel(r'$\chi_{3}$')
ax1.set_ylabel(r'Energy kcal/mol')
ax1.set_title(r'Energy vs $\chi_{3}$')
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 30))

plt.show()


In [ ]:
#
TORMAX = 180 # for left handed (Chi3 < 0)
fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)
ax1 = plt.subplot(111)

x = SS_df['chi3']
y = SS_df['ca_distance']

ymax = 8.0
plt.hexbin(x, y, gridsize=80, cmap='nipy_spectral')
ax1.set_xlim(TORMIN-1, TORMAX)
ax1.set_ylim(y.min(), y.max())
ax1.set_xlabel(r'$\chi_{3}$')
ax1.set_ylabel(r'C${\alpha}$ Distance ($\AA$)')
ax1.set_title(r'C${\alpha}$ Distance vs $\chi_{3}$')
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 30))

plt.show()


## Disulfide Clustering
If we could cluster the disulfides by their conformation we would be able to find conformational families; common structural motifs the recur within protein structures. The ```Scikit-Learn``` package offers a variety of clustering techniques. I explore some of them in the cells below.

### Gaussian Mixture

In [ ]:
from sklearn.mixture import GaussianMixture
n_clusters = 4

_cols = ['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'torsion_length', 'energy', 'ca_distance']

tor_df = SS_df[_cols]
tor_df.head(1)
gmm_model = GaussianMixture(n_components=n_clusters)
gmm_model.fit(tor_df)
cluster_labels = gmm_model.predict(tor_df)
X = pd.DataFrame(tor_df)
X['cluster'] = cluster_labels
for k in range(n_clusters):
    data = X[X['cluster'] == k]
    plt.scatter(data['torsion_length'], data['energy'], s=1)

plt.show()



In [ ]:
tor_df.head()

### Spectral Clustering

In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import SpectralClustering
import seaborn as sns

_cols = ['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'torsion_length', 'energy']
#cols = ['ca_distance', 'chi3', 'energy', 'torsion_length']

# tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5']].copy()

tor_df = SS_df[_cols].copy()

X = tor_df
n_clusters = 4

scm_model = SpectralClustering(n_clusters=n_clusters, random_state=25,
                               n_neighbors=16, affinity='nearest_neighbors')
# takes 51 min with full dataset
X['cluster'] = scm_model.fit_predict(X)

fig, ax = plt.subplots()
ax.set(title='Spectral Clustering')
sns.scatterplot(y='energy', x='torsion_length', data=X, hue='cluster', ax=ax, size=2)


### Affinity Propagation

In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import AffinityPropagation
import seaborn as sns
n_clusters = 6
custom_palette = sns.color_palette("Dark2", n_clusters)


_cols = ['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'torsion_length']
tor_df = SS_df[_cols].copy()

X = tor_df.copy()

aff_model = AffinityPropagation(max_iter=300, random_state=25)
# takes 51 min with full dataset
X['cluster'] = aff_model.fit_predict(X)

fig, ax = plt.subplots()
ax.set(title='Affinity Propagation')
#sns.scatterplot(x='chi1', y='chi5', data=X, hue='cluster', ax=ax, size=2, palette=custom_palette)



In [ ]:
fig, ax = plt.subplots()
ax.set(title='Affinity Propagation')
sns.scatterplot(x='chi1', y='chi5', data=X, hue='cluster', ax=ax, size=2)


### Agglomerative Clustering

In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import AgglomerativeClustering

n_clusters = 4
custom_palette = sns.color_palette("Dark2", n_clusters)

tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'torsion_length']].copy()
X = tor_df.copy()

agg_model = AgglomerativeClustering(n_clusters=n_clusters)

X['cluster'] = agg_model.fit_predict(X[['torsion_length']])

fig, ax = plt.subplots()
ax.set(title='Agglomerative Clustering')
sns.scatterplot(x='chi1', y='chi5', data=X, hue='cluster', ax=ax, size=2, palette=custom_palette)


In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import AgglomerativeClustering

n_clusters = 8
custom_palette = sns.color_palette("Dark2", n_clusters)


tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'energy', 'ca_distance']].copy()
X = tor_df.copy()

agg_model = AgglomerativeClustering(n_clusters=n_clusters)

X['cluster'] = agg_model.fit_predict(X[['energy']])

fig, ax = plt.subplots()
ax.set(title='Agglomerative Clustering')
sns.scatterplot(x='chi1', y='energy', data=X, hue='cluster', ax=ax, size=2, palette=custom_palette)


In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import AgglomerativeClustering

n_clusters = 8
custom_palette = sns.color_palette("Dark2", n_clusters)


tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'ca_distance', 'torsion_length']].copy()
X = tor_df.copy()

agg_model = AgglomerativeClustering(n_clusters=n_clusters)

X['cluster'] = agg_model.fit_predict(X)

fig, ax = plt.subplots()
ax.set(title='Agglomerative Clustering')
sns.scatterplot(x='chi1', y='chi2', data=X, hue='cluster', ax=ax, size=1, palette=custom_palette)


In [ ]:
# takes over an hour for full dataset
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns

n_clusters = 4
custom_palette = sns.color_palette("Dark2", n_clusters)


tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'torsion_length']].copy()
X = tor_df.copy()

agg_model = AgglomerativeClustering(n_clusters=n_clusters)

X['cluster'] = agg_model.fit_predict(X)

fig, ax = plt.subplots()
ax.set(title='Agglomerative Clustering')
sns.scatterplot(x='chi1', y='chi3', data=X, hue='cluster', ax=ax, size=2, palette=custom_palette)


In [ ]:
# chatgpt output

from sklearn.cluster import AgglomerativeClustering

def cluster_by_family(data, n_clusters=3):
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agg_clustering.fit_predict(data)
    return labels


In [ ]:
labels = {'value': 'Chi3', 'variable': 'Variable'}

x = SS_df['chi3']
y = SS_df['energy']
df = SS_df[['chi3', 'energy']].copy()

energy_hist = numpy.histogram2d(x, y=y, bins=360)
energy_hist = energy_hist

fig = px.histogram(df, labels=labels)

fig.show()

In [ ]:
cols = ['chi1', 'chi5']
px.histogram(SS_df[cols], labels=labels)

In [ ]:
cols = ['chi2', 'chi4']
px.histogram(SS_df[cols], labels=labels)

In [ ]:
px.histogram(SS_df['chi3'], labels=labels)

## Examining Disulfide Conformational Families
Richardson et al characterized two principal conformational families for Disulfide bonds, a *Left Handed Spiral* with dihedral angles around, -60, -90, -90, -90, -60 and a *Right Handed Hook* with dihedrals, -60, 120, 90, -50, -60. My research found another class, the *Left Handed Hook* with dihedrals x, x, x, x, x. We can easily examine the representative neighbors of these families using one of the nearest neighbor functions available through the ```Disulfide``` class.

In [ ]:

# retrieve the torsions dataframe
from proteusPy.Disulfide import Torsion_DF_Cols

_SSdf = PDB_SS.getTorsions()

# there are a few structures with bad SSBonds. Their
# CA distances are > 7.0. We remove them from consideration
# below

_near = _SSdf['ca_distance'] < 9.0

# entire database
SS_df = _SSdf[_near]

SS_df = SS_df[Torsion_DF_Cols].copy()
SS_df.describe()

sslist = PDB_SS.SSList
LHS_neighbors = sslist.nearest_neighbors(-60, -90, -90, -90, -60, 15.0)
LHS_neighbors.length

In [ ]:
LHS_neighbors.display_overlay(light=False)

In [ ]:
sslist = PDB_SS.SSList
RHH_neighbors = sslist.nearest_neighbors(-60, 120, 90, -50, -60, 30)
RHH_neighbors.length

In [ ]:
RHH_neighbors.display_overlay(light=False)

## Distribution of Backbone Dihedral Angles $\phi - \psi$
A protein's backbone conformation is defined by the dihdedral angles $\phi$ and $\psi$. The ```proteusPy.Disulfide.ExtractDisulfides()``` function computes these at time of Disulfide extraction.

In [ ]:
GRIDSIZE=40

x = SS_df['phi_prox']
y = SS_df['psi_prox']

x2 = SS_df['phi_dist']
y2 = SS_df['psi_dist']

plt.rcParams.update({'font.size': 8})
plt.rcParams['text.usetex'] = True

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)

fig.suptitle(r'RCSB Disulfide Bond $\phi - \psi$ Dihedral Angles', fontsize=8)

ax1, ax2 = fig.subplots(1, 2, sharey=False)

ax1.tick_params(axis='both', which='major', labelsize=6)
ax2.tick_params(axis='both', which='major', labelsize=6)

ax1.set_xlim(TORMIN-1, TORMAX+1)
ax1.set_ylim(TORMIN-1, TORMAX+1)
ax1.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax1.set_xlabel(r'$\phi$ Proximal')
ax1.set_ylabel(r'$\psi$')

ax1.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')

ax2.set_xlim(TORMIN, TORMAX)
ax2.set_ylim(-180, 180)
ax2.xaxis.set_ticks(numpy.arange(TORMIN-1, TORMAX+1, 60))

ax2.set_xlabel(r'$\phi$ Distal')
ax2.hexbin(x2, y2, gridsize=GRIDSIZE, cmap='nipy_spectral')

plt.show()


## Summary
Conformational analysis of 294,222 disulfide bonds in 36,362 proteins contained in the RCSB confirms the predominant conformational classes first described in my initial analysis:
- Left-Handed Spiral
- Right-Handed Hook
- Left-Handed Spiral
  

# Appendix - Data Management

## Downloading Disulfide-containing Structures From RCSB
I started the process by creating a query on (RCSB)[https://www.rcsb.org] searching for all proteins containing one or more disulfide bonds. The resulting 38214 PDB IDs were downloaded using the ```DownloadDisulfides``` program contained in the ```programs``` folder. This took approximately 12 hours, and occupies approximately 30GB.

## Extract the Disulfides from the PDB files
The function ``Extract_Disulfides()`` processes all the .ent files in ``PDB_DIR`` and creates two .pkl files representing the Disulfide bonds contained in the scanned directory. In addition, a .csv file containing problem IDs is written if any are found. The .pkl files are consumed by the ``DisulfideLoader`` class and are considered private. You'll see numerous warnings during the scan. Files that are unparsable are removed and their IDs are logged to the problem_id.csv file. The default file locations are stored in the file globals.py and are the used by ``DisulfideExtractor()`` in the absence of arguments passed. The Disulfide parser is very stringent and will reject disulfide bonds with missing atoms or disordered atoms.


Outputs are saved in ``DATA_DIR``:
1) ``SS_PICKLE_FILE``: The ``DisulfideList`` of ``Disulfide`` objects initialized from the PDB file scan, needed by the ``DisulfideLoader()`` class.
2) ``SS_DICT_PICKLE_FILE``: the ``Dict Disulfide`` objects also needed by the ``DisulfideLoader()`` class
3) ``PROBLEM_ID_FILE``: a .csv containining the problem ids.

In general, the process only needs to be run once for a full scan. Setting the ``numb`` argument to -1 scans the entire directory. Entering a positive number allows parsing a subset of the dataset, which is useful when debugging. Setting ``verbose`` enables verbose messages. Setting ``quiet`` to ``True`` disables all warnings.

NB: A extraction of the initial disulfide bond-containing files (> 36000 files) takes about 1.25 hours on a 2020 MacbookPro with M1 Pro chip, 16GB RAM, 1TB SSD. The resulting .pkl files consume approximately 1GB of disk space, and equivalent RAM used when loaded.

## Data Cleaning
* Parsing PDB files
* Extracting disulfides
* Removing redundant disulfides

## Load the Disulfide Data
Now that the Disulfides have been extracted and the Disulfide .pkl files have been created we can load them into memory using the DisulfideLoader() class. This class stores the Disulfides internally as a DisulfideList and a dict. Array indexing operations including slicing have been overloaded, enabling straightforward access to the Disulfide bonds, both in aggregate and by residue. After loading the .pkl files the Class creates a Pandas ``DataFrame`` object consisting of the Disulfide ID, all sidechain dihedral angles, the local coordinates for the Disulfide and the computed Disulfide bond torsional energy.

NB: Loading the data takes 3.5 minutes on my MacbookPro. Be patient if it seems to take a long time to load.

The ```Disulfide``` and ```DisulfideList``` classes include rendering capabiities using the excellent PyVista interface to the VTK package. (http://pyvista.org). The following cell displays the first Disulfide bond in the database in ball-and stick style. Atoms are colored by atom type:
- Grey = Carbon
- Blue = Nitrogen
- Red = Oxygen
- Yellow = Sulfur
- White = Previous residue carbonyl carbon and next residue amino Nitrogen. (more on this below).

We can load the database and display the first disulfide as shown below. The molecular display is *interactive*; select-drag to rotate, mousewheel to zoom. The X-Y-Z widget in the window upper right allows orientation against the X, Y and Z axes. The window title provides information about the disulfide rendered:
* Disulfide source ID
* Disulfide name, which embodies the source id, the proximal residue number and the distal residue number.
* Disulfide approximate torsional energy (kcal/mol)
* Disulfide $C_\alpha-C_\alpha$ distance $\AA$
* Disulfide torsion length (5-dimensional vector length) (degrees)



# Publications
* https://doi.org/10.1021/bi00368a023
* https://doi.org/10.1021/bi00368a024
* https://doi.org/10.1016/0092-8674(92)90140-8
* http://dx.doi.org/10.2174/092986708783330566